# CS2316 Final Project — Phase II  
### Data Collection, Cleaning, Standardization, and Merge Preparation

In this phase, the objective is to implement an end-to-end ETL pipeline for the three required datasets:

1) **Downloaded dataset** — DOT Consumer Airfare Report (Table 1A CSV)  
2) **HTML-sourced dataset** — BTS Baggage Fee revenues (web-scraped or link-extracted Excel)  
3) **API/JSON dataset** — EIA WTI crude oil prices (via REST API)

Additionally, an optional **FRED CPI JSON dataset** will be used for inflation normalization.

The tasks for Phase II include:
- Programmatically retrieving all required datasets (read/download/API/scrape)
- Parsing into DataFrames
- Cleaning and standardizing fields (date alignment to quarters, carrier naming, currency inflation-adjustment)
- Creating at least 3–5 documented inconsistencies and resolving them in-code
- Exporting all cleaned datasets to new files
- Returning the cleaned DataFrames in the notebook
- Preparing the merged structure so Phase III analysis can be done immediately

Deliverable is a fully executed Jupyter notebook showing function-based ETL for each dataset, with cleaned exports and documented inconsistencies visible in the output.

This part is written by: Jackson O'Connell